### Manipulation of Peaks de novo results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Peaks de novo results (.csv) of PTM-optimized database searches > ALC 50%

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported PeaksDN search results CSVs into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt


In [2]:
ls

231/  233/  243/  273/  278/  378/


In [3]:
ls 378

ETNP_SKQ17_DENOVO_166_378-100m-trap_DN50_15ppm.csv
ETNP_SKQ17_DENOVO_166_378-100m-trap_DN50_15ppm.xml


In [4]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

## 378: 100 m +P filtered on 0.3 um GF-75

In [6]:
# read the CSV into a dataframe using the pandas read_csv function
peaks378_50 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP_SKQ17_DENOVO_166_378-100m-trap_DN50_15ppm.csv")

# also make a dataframe but keep only peptides >80% ALC
peaks378_80 = peaks378_50.loc[peaks378_50['ALC (%)'] >= 80].copy()

# how many de novo sequence candidates >50% ALC?
print("# redundant Peaks peptides >50% ALC in dataframe", len(peaks378_50))
# how many de novo sequence candidates >80ALC?
print("# redundant Peaks peptides >80% ALC in dataframe", len(peaks378_80))

#look at the dataframe
peaks378_50.head()

# redundant Peaks peptides >50% ALC in dataframe 1575
# redundant Peaks peptides >80% ALC in dataframe 459


,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,4,4564,20181214_378_etnp2017_100m_trap.raw,VVEEAPK,7,99,7,386.2155,2,17.85,800000.0,770.4174,-1.1,NaN,99 99 100 100 99 99 99,VVEEAPK,CID
1,4,4574,20181214_378_etnp2017_100m_trap.raw,SLEAESR,7,98,7,396.1982,2,17.88,513000.0,790.3821,-0.4,NaN,98 99 100 99 99 98 97,SLEAESR,CID
2,4,3812,20181214_378_etnp2017_100m_trap.raw,TEEAVEEK,8,98,8,467.7217,2,15.60,232000.0,933.4291,-0.2,NaN,99 100 100 99 97 99 100 94,TEEAVEEK,CID
3,4,4982,20181214_378_etnp2017_100m_trap.raw,EEDLER,6,98,6,395.6826,2,19.14,245000.0,789.3504,0.3,NaN,100 100 98 98 99 95,EEDLER,CID
4,4,11187,20181214_378_etnp2017_100m_trap.raw,FEELVEK,7,98,7,447.2337,2,41.12,815000.0,892.4542,-1.4,NaN,96 99 100 100 97 99 96,FEELVEK,CID


In [7]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks378_50['A'] = peaks378_50['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks378_50['C'] = peaks378_50['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks378_50['D'] = peaks378_50['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks378_50['E'] = peaks378_50['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks378_50['F'] = peaks378_50['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks378_50['G'] = peaks378_50['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks378_50['H'] = peaks378_50['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks378_50 output, there will be no isoleucines (they're lumped in with leucines)
peaks378_50['I'] = peaks378_50['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks378_50['K'] = peaks378_50['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks378_50['L'] = peaks378_50['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks378_50['M'] = peaks378_50['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks378_50['N'] = peaks378_50['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks378_50['P'] = peaks378_50['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks378_50['Q'] = peaks378_50['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks378_50['R'] = peaks378_50['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks378_50['S'] = peaks378_50['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks378_50['T'] = peaks378_50['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks378_50['V'] = peaks378_50['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks378_50['W'] = peaks378_50['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks378_50['Y'] = peaks378_50['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks378_50['c-carb'] = peaks378_50['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks378_50['m-oxid'] = peaks378_50['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks378_50['n-deam'] = peaks378_50['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks378_50['q-deam'] = peaks378_50['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks378_50['k-hydr'] = peaks378_50['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
# no more P hydroxylation in final searches
# peaks378_50['p-hydr'] = peaks378_50['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks378_50['r-meth'] = peaks378_50['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks378_50['stripped peptide'] = peaks378_50['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks378_50['stripped length'] = peaks378_50['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks378_50['ptm-total'] = peaks378_50['c-carb'] + peaks378_50['m-oxid'] + peaks378_50['n-deam'] \
+ peaks378_50['q-deam'] + peaks378_50['k-hydr'] + peaks378_50['r-meth']

# calculate NAAF numerator for each peptide k
peaks378_50['NAAF num.'] = peaks378_50['Area'] / peaks378_50['stripped length']

# write modified dataframe to new csv file
peaks378_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN50.csv")


# check out the results
peaks378_50.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,c-carb,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,4,4564,20181214_378_etnp2017_100m_trap.raw,VVEEAPK,7,99,7,386.2155,2,17.85,...,0,0,0,0,0,0,VVEEAPK,7,0,114285.714286
1,4,4574,20181214_378_etnp2017_100m_trap.raw,SLEAESR,7,98,7,396.1982,2,17.88,...,0,0,0,0,0,0,SLEAESR,7,0,73285.714286
2,4,3812,20181214_378_etnp2017_100m_trap.raw,TEEAVEEK,8,98,8,467.7217,2,15.60,...,0,0,0,0,0,0,TEEAVEEK,8,0,29000.000000
3,4,4982,20181214_378_etnp2017_100m_trap.raw,EEDLER,6,98,6,395.6826,2,19.14,...,0,0,0,0,0,0,EEDLER,6,0,40833.333333
4,4,11187,20181214_378_etnp2017_100m_trap.raw,FEELVEK,7,98,7,447.2337,2,41.12,...,0,0,0,0,0,0,FEELVEK,7,0,116428.571429


In [8]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks378_50['A'].sum(),
        'C': peaks378_50['C'].sum(),
        'D': peaks378_50['D'].sum(),
        'E': peaks378_50['E'].sum(),
        'F': peaks378_50['F'].sum(),
        'G': peaks378_50['G'].sum(),
        'H': peaks378_50['H'].sum(),
        'I': peaks378_50['I'].sum(),
        'K': peaks378_50['K'].sum(),
        'L': peaks378_50['L'].sum(),
        'M': peaks378_50['M'].sum(),
        'N': peaks378_50['N'].sum(),
        'P': peaks378_50['P'].sum(),
        'Q': peaks378_50['Q'].sum(),
        'R': peaks378_50['R'].sum(),
        'S': peaks378_50['S'].sum(),
        'T': peaks378_50['T'].sum(),
        'V': peaks378_50['V'].sum(),
        'W': peaks378_50['W'].sum(),
        'Y': peaks378_50['Y'].sum(),
        'c-carb': peaks378_50['c-carb'].sum(),
        'm-oxid': peaks378_50['m-oxid'].sum(),
        'n-deam': peaks378_50['n-deam'].sum(),
        'q-deam': peaks378_50['q-deam'].sum(),
        'k-hydr': peaks378_50['k-hydr'].sum(),
        'r-meth': peaks378_50['r-meth'].sum(),
        'Total area': peaks378_50['Area'].sum(),
        'Total length': peaks378_50['stripped length'].sum()
       }

totalpeaks378_50 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',\
                                               'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', \
                                               'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth', \
                                               'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks378_50['% C w/ carb'] = totalpeaks378_50['c-carb'] / totalpeaks378_50['C'] 

# calculate percentage of M's that are oxidized
totalpeaks378_50['% M w/ oxid'] = totalpeaks378_50['m-oxid'] / totalpeaks378_50['M'] 

# calculate percentage of N's that are deamidated
totalpeaks378_50['% N w/ deam'] = totalpeaks378_50['n-deam'] / totalpeaks378_50['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks378_50['% Q w/ deam'] = totalpeaks378_50['q-deam'] / totalpeaks378_50['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks378_50['% K w/ hydr'] = totalpeaks378_50['k-hydr'] / totalpeaks378_50['K'] 

# calculate percentage of P's that are hydroxylated
#totalpeaks378_50['% P w/ hydr'] = totalpeaks378_50['p-hydr'] / totalpeaks378_50['K'] 

# calculate percentage of R's that are methylated
totalpeaks378_50['% R w/ meth'] = totalpeaks378_50['r-meth'] / totalpeaks378_50['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks378_50['NAAF denom.'] = totalpeaks378_50['Total area'] / totalpeaks378_50['Total length']

# write modified dataframe to new txt file
totalpeaks378_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN50_totals.csv")

totalpeaks378_50.head()

,A,C,D,E,F,G,H,I,K,L,...,r-meth,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% R w/ meth,NAAF denom.
sample total,1120,240,659,1097,289,542,242,0,1236,1052,...,325,1.965749e+09,11610,1.0,0.476,0.256983,0.04,0.182039,0.349087,169315.155492


In [9]:
# use the calculated NAAF factor (in totalpeaks378 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF50 = 169315.155492

# use NAAF >50% ALC to get NAAF
peaks378_50['NAAF factor'] = (peaks378_50['NAAF num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks378_AA50 = peaks378_50[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                             'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', \
                             'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF50 factor by the AAs to normalize its abundance by peak area and peptide length

peaks378_AA50['A-NAAF50'] = peaks378_AA50['A'] * peaks378_50['NAAF factor']
peaks378_AA50['C-NAAF50'] = peaks378_AA50['C'] * peaks378_50['NAAF factor']
peaks378_AA50['D-NAAF50'] = peaks378_AA50['D'] * peaks378_50['NAAF factor']
peaks378_AA50['E-NAAF50'] = peaks378_AA50['E'] * peaks378_50['NAAF factor']
peaks378_AA50['F-NAAF50'] = peaks378_AA50['F'] * peaks378_50['NAAF factor']
peaks378_AA50['G-NAAF50'] = peaks378_AA50['G'] * peaks378_50['NAAF factor']
peaks378_AA50['H-NAAF50'] = peaks378_AA50['H'] * peaks378_50['NAAF factor']
peaks378_AA50['I-NAAF50'] = peaks378_AA50['I'] * peaks378_50['NAAF factor']
peaks378_AA50['K-NAAF50'] = peaks378_AA50['K'] * peaks378_50['NAAF factor']
peaks378_AA50['L-NAAF50'] = peaks378_AA50['L'] * peaks378_50['NAAF factor']
peaks378_AA50['M-NAAF50'] = peaks378_AA50['M'] * peaks378_50['NAAF factor']
peaks378_AA50['N-NAAF50'] = peaks378_AA50['N'] * peaks378_50['NAAF factor']
peaks378_AA50['P-NAAF50'] = peaks378_AA50['P'] * peaks378_50['NAAF factor']
peaks378_AA50['Q-NAAF50'] = peaks378_AA50['Q'] * peaks378_50['NAAF factor']
peaks378_AA50['R-NAAF50'] = peaks378_AA50['R'] * peaks378_50['NAAF factor']
peaks378_AA50['S-NAAF50'] = peaks378_AA50['S'] * peaks378_50['NAAF factor']
peaks378_AA50['T-NAAF50'] = peaks378_AA50['T'] * peaks378_50['NAAF factor']
peaks378_AA50['V-NAAF50'] = peaks378_AA50['V'] * peaks378_50['NAAF factor']
peaks378_AA50['W-NAAF50'] = peaks378_AA50['W'] * peaks378_50['NAAF factor']
peaks378_AA50['Y-NAAF50'] = peaks378_AA50['Y'] * peaks378_50['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaks378_AA50['ccarb-NAAF50'] = peaks378_AA50['c-carb'] * peaks378_AA50['NAAF factor']
peaks378_AA50['moxid-NAAF50'] = peaks378_AA50['m-oxid'] * peaks378_AA50['NAAF factor']
peaks378_AA50['ndeam-NAAF50'] = peaks378_AA50['n-deam'] * peaks378_AA50['NAAF factor']
peaks378_AA50['qdeam-NAAF50'] = peaks378_AA50['q-deam'] * peaks378_AA50['NAAF factor']
peaks378_AA50['khydr-NAAF50'] = peaks378_AA50['k-hydr'] * peaks378_AA50['NAAF factor']
peaks378_AA50['rmeth-NAAF50'] = peaks378_AA50['r-meth'] * peaks378_AA50['NAAF factor']

# write the dataframe to a new csv
peaks378_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN50_NAAF.csv")

peaks378_AA50.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50,khydr-NAAF50,rmeth-NAAF50
0,VVEEAPK,0.674988,1,0,0,2,0,0,0,0,...,0.000000,1.349976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SLEAESR,0.432836,1,0,0,2,0,0,0,0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TEEAVEEK,0.171278,1,0,0,4,0,0,0,0,...,0.171278,0.171278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EEDLER,0.241168,0,0,1,3,0,0,0,0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FEELVEK,0.687644,0,0,0,3,1,0,0,0,...,0.000000,0.687644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet378 PeaksDN results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaks378_AA50['NAAF factor'].sum(),
        'A-NAAF': peaks378_AA50['A-NAAF50'].sum(),
        'C-NAAF': peaks378_AA50['C-NAAF50'].sum(),
        'D-NAAF': peaks378_AA50['D-NAAF50'].sum(),
        'E-NAAF': peaks378_AA50['E-NAAF50'].sum(),
        'F-NAAF': peaks378_AA50['F-NAAF50'].sum(),
        'G-NAAF': peaks378_AA50['G-NAAF50'].sum(),
        'H-NAAF': peaks378_AA50['H-NAAF50'].sum(),
        'I-NAAF': peaks378_AA50['I-NAAF50'].sum(),
        'K-NAAF': peaks378_AA50['K-NAAF50'].sum(),
        'L-NAAF': peaks378_AA50['L-NAAF50'].sum(),
        'M-NAAF': peaks378_AA50['M-NAAF50'].sum(),
        'N-NAAF': peaks378_AA50['N-NAAF50'].sum(),
        'P-NAAF': peaks378_AA50['P-NAAF50'].sum(),
        'Q-NAAF': peaks378_AA50['Q-NAAF50'].sum(),
        'R-NAAF': peaks378_AA50['R-NAAF50'].sum(),
        'S-NAAF': peaks378_AA50['S-NAAF50'].sum(),
        'T-NAAF': peaks378_AA50['T-NAAF50'].sum(),
        'V-NAAF': peaks378_AA50['V-NAAF50'].sum(),
        'W-NAAF': peaks378_AA50['W-NAAF50'].sum(),
        'Y-NAAF': peaks378_AA50['Y-NAAF50'].sum(),
        'C-carb-NAAF': peaks378_AA50['ccarb-NAAF50'].sum(),
        'M-oxid-NAAF': peaks378_AA50['moxid-NAAF50'].sum(),
        'N-deam-NAAF': peaks378_AA50['ndeam-NAAF50'].sum(),
        'Q-deam-NAAF': peaks378_AA50['qdeam-NAAF50'].sum(),
        'K-hydr-NAAF': peaks378_AA50['khydr-NAAF50'].sum(),
        'R-meth-NAAF': peaks378_AA50['rmeth-NAAF50'].sum()
       }

totalpeaks378_AA50 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF', 'K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, P, and Rs


totalpeaks378_AA50['% C w/ carb. NAAF'] = totalpeaks378_AA50['C-carb-NAAF'] / totalpeaks378_AA50['C-NAAF']
totalpeaks378_AA50['% M w/ oxid. NAAF'] = totalpeaks378_AA50['M-oxid-NAAF'] / totalpeaks378_AA50['M-NAAF']
totalpeaks378_AA50['% N w/ deam. NAAF'] = totalpeaks378_AA50['N-deam-NAAF'] / totalpeaks378_AA50['N-NAAF']
totalpeaks378_AA50['% Q w/ deam. NAAF'] = totalpeaks378_AA50['Q-deam-NAAF'] / totalpeaks378_AA50['Q-NAAF']
totalpeaks378_AA50['% K w/ hydr. NAAF'] = totalpeaks378_AA50['K-hydr-NAAF'] / totalpeaks378_AA50['K-NAAF']
totalpeaks378_AA50['% R w/ meth. NAAF'] = totalpeaks378_AA50['R-meth-NAAF'] / totalpeaks378_AA50['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks378_AA50['NAAF check'] = totalpeaks378_AA50['NAAF'] / 169315.155492

# write modified dataframe to new txt file, same name + totals
totalpeaks378_AA50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_DN50_NAAF_totals.csv")

totalpeaks378_AA50.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,Q-deam-NAAF,K-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,1455.090524,1281.467584,81.100199,246.599754,766.725937,212.767573,369.096463,46.262004,0.0,504.926046,...,17.918274,70.749043,669.06165,1.0,0.481743,0.428399,0.101171,0.140118,0.552833,0.008594


### Same process but for de novo peptide >80 % ALC:

In [11]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks378_80['A'] = peaks378_80['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks378_80['C'] = peaks378_80['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks378_80['D'] = peaks378_80['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks378_80['E'] = peaks378_80['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks378_80['F'] = peaks378_80['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks378_80['G'] = peaks378_80['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks378_80['H'] = peaks378_80['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks378_80 output, there will be no isoleucines (they're lumped in with leucines)
peaks378_80['I'] = peaks378_80['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks378_80['K'] = peaks378_80['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks378_80['L'] = peaks378_80['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks378_80['M'] = peaks378_80['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks378_80['N'] = peaks378_80['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks378_80['P'] = peaks378_80['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks378_80['Q'] = peaks378_80['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks378_80['R'] = peaks378_80['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks378_80['S'] = peaks378_80['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks378_80['T'] = peaks378_80['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks378_80['V'] = peaks378_80['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks378_80['W'] = peaks378_80['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks378_80['Y'] = peaks378_80['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks378_80['c-carb'] = peaks378_80['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks378_80['m-oxid'] = peaks378_80['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks378_80['n-deam'] = peaks378_80['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks378_80['q-deam'] = peaks378_80['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of hydroxylated K's in each peptide
peaks378_80['k-hydr'] = peaks378_80['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of hydroxylated P's in each peptide
#peaks378_80['p-hydr'] = peaks378_80['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks378_80['r-meth'] = peaks378_80['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# create a column with 'stripped' peptide sequences using strip
peaks378_80['stripped peptide'] = peaks378_80['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks378_80['stripped length'] = peaks378_80['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks378_80['ptm-total'] = peaks378_80['c-carb'] + peaks378_80['m-oxid'] + peaks378_80['n-deam'] + \
peaks378_80['q-deam'] + peaks378_80['k-hydr'] + peaks378_80['r-meth']

# calculate NAAF numerator for each peptide k
peaks378_80['NAAF num.'] = peaks378_80['Area'] / peaks378_80['stripped length']

# write modified dataframe to new csv file
peaks378_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN80.csv")

# check out the results
peaks378_80.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,c-carb,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,ptm-total,NAAF num.
0,4,4564,20181214_378_etnp2017_100m_trap.raw,VVEEAPK,7,99,7,386.2155,2,17.85,...,0,0,0,0,0,0,VVEEAPK,7,0,114285.714286
1,4,4574,20181214_378_etnp2017_100m_trap.raw,SLEAESR,7,98,7,396.1982,2,17.88,...,0,0,0,0,0,0,SLEAESR,7,0,73285.714286
2,4,3812,20181214_378_etnp2017_100m_trap.raw,TEEAVEEK,8,98,8,467.7217,2,15.60,...,0,0,0,0,0,0,TEEAVEEK,8,0,29000.000000
3,4,4982,20181214_378_etnp2017_100m_trap.raw,EEDLER,6,98,6,395.6826,2,19.14,...,0,0,0,0,0,0,EEDLER,6,0,40833.333333
4,4,11187,20181214_378_etnp2017_100m_trap.raw,FEELVEK,7,98,7,447.2337,2,41.12,...,0,0,0,0,0,0,FEELVEK,7,0,116428.571429


In [12]:
# made a new dataframe that contains the sums of certain columns in modified 
#peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks378_80['A'].sum(),
        'C': peaks378_80['C'].sum(),
        'D': peaks378_80['D'].sum(),
        'E': peaks378_80['E'].sum(),
        'F': peaks378_80['F'].sum(),
        'G': peaks378_80['G'].sum(),
        'H': peaks378_80['H'].sum(),
        'I': peaks378_80['I'].sum(),
        'K': peaks378_80['K'].sum(),
        'L': peaks378_80['L'].sum(),
        'M': peaks378_80['M'].sum(),
        'N': peaks378_80['N'].sum(),
        'P': peaks378_80['P'].sum(),
        'Q': peaks378_80['Q'].sum(),
        'R': peaks378_80['R'].sum(),
        'S': peaks378_80['S'].sum(),
        'T': peaks378_80['T'].sum(),
        'V': peaks378_80['V'].sum(),
        'W': peaks378_80['W'].sum(),
        'Y': peaks378_80['Y'].sum(),
        'c-carb': peaks378_80['c-carb'].sum(),
        'm-oxid': peaks378_80['m-oxid'].sum(),
        'n-deam': peaks378_80['n-deam'].sum(),
        'q-deam': peaks378_80['q-deam'].sum(),
        'k-hydr': peaks378_80['k-hydr'].sum(),
        'r-meth': peaks378_80['r-meth'].sum(),
        'Total area': peaks378_80['Area'].sum(),
        'Total length': peaks378_80['stripped length'].sum()
       }

totalpeaks378_80 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',\
                                               'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb',\
                                               'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth',\
                                               'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks378_80['% C w/ carb'] = totalpeaks378_80['c-carb'] / totalpeaks378_80['C'] 

# calculate percentage of M's that are oxidized
totalpeaks378_80['% M w/ oxid'] = totalpeaks378_80['m-oxid'] / totalpeaks378_80['M'] 

# calculate percentage of N's that are deamidated
totalpeaks378_80['% N w/ deam'] = totalpeaks378_80['n-deam'] / totalpeaks378_80['N'] 

# calculate percentage of Q's that are deamidated
totalpeaks378_80['% Q w/ deam'] = totalpeaks378_80['q-deam'] / totalpeaks378_80['Q'] 

# calculate percentage of K's that are hydroxylated
totalpeaks378_80['% K w/ hydr'] = totalpeaks378_80['k-hydr'] / totalpeaks378_80['K'] 

# calculate percentage of P's that are hydroxylated
#totalpeaks378_80['% P w/ hydr'] = totalpeaks378_80['p-hydr'] / totalpeaks378_80['K'] 

# calculate percentage of R's that are methylated
totalpeaks378_80['% R w/ meth'] = totalpeaks378_80['r-meth'] / totalpeaks378_80['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks378_80['NAAF denom.'] = totalpeaks378_80['Total area'] / totalpeaks378_80['Total length']

# write modified dataframe to new txt file
totalpeaks378_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN80_totals.csv")

totalpeaks378_80.head()

,A,C,D,E,F,G,H,I,K,L,...,r-meth,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% R w/ meth,NAAF denom.
sample total,369,23,209,469,79,144,19,0,301,391,...,67,1.557270e+09,3533,1.0,0.561644,0.304183,0.066176,0.07309,0.25283,440778.253609


In [13]:
# use the calculated NAAF factor (in totalpeaks378 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we will with Comet results

NAAF80 = 440778.253609

# use NAAF >80% ALC to get NAAF
peaks378_80['NAAF factor'] = (peaks378_80['NAAF num.'])/NAAF80

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaks378_AA80 = peaks378_80[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                             'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                             'n-deam', 'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF80 factor by the AAs to normalize its abundance by peak area and peptide length

peaks378_AA80['A-NAAF80'] = peaks378_AA80['A'] * peaks378_80['NAAF factor']
peaks378_AA80['C-NAAF80'] = peaks378_AA80['C'] * peaks378_80['NAAF factor']
peaks378_AA80['D-NAAF80'] = peaks378_AA80['D'] * peaks378_80['NAAF factor']
peaks378_AA80['E-NAAF80'] = peaks378_AA80['E'] * peaks378_80['NAAF factor']
peaks378_AA80['F-NAAF80'] = peaks378_AA80['F'] * peaks378_80['NAAF factor']
peaks378_AA80['G-NAAF80'] = peaks378_AA80['G'] * peaks378_80['NAAF factor']
peaks378_AA80['H-NAAF80'] = peaks378_AA80['H'] * peaks378_80['NAAF factor']
peaks378_AA80['I-NAAF80'] = peaks378_AA80['I'] * peaks378_80['NAAF factor']
peaks378_AA80['K-NAAF80'] = peaks378_AA80['K'] * peaks378_80['NAAF factor']
peaks378_AA80['L-NAAF80'] = peaks378_AA80['L'] * peaks378_80['NAAF factor']
peaks378_AA80['M-NAAF80'] = peaks378_AA80['M'] * peaks378_80['NAAF factor']
peaks378_AA80['N-NAAF80'] = peaks378_AA80['N'] * peaks378_80['NAAF factor']
peaks378_AA80['P-NAAF80'] = peaks378_AA80['P'] * peaks378_80['NAAF factor']
peaks378_AA80['Q-NAAF80'] = peaks378_AA80['Q'] * peaks378_80['NAAF factor']
peaks378_AA80['R-NAAF80'] = peaks378_AA80['R'] * peaks378_80['NAAF factor']
peaks378_AA80['S-NAAF80'] = peaks378_AA80['S'] * peaks378_80['NAAF factor']
peaks378_AA80['T-NAAF80'] = peaks378_AA80['T'] * peaks378_80['NAAF factor']
peaks378_AA80['V-NAAF80'] = peaks378_AA80['V'] * peaks378_80['NAAF factor']
peaks378_AA80['W-NAAF80'] = peaks378_AA80['W'] * peaks378_80['NAAF factor']
peaks378_AA80['Y-NAAF80'] = peaks378_AA80['Y'] * peaks378_80['NAAF factor']

# multiply the NAAF80 factor by the PTMs normalize its abundance by peak area and peptide length

peaks378_AA80['ccarb-NAAF80'] = peaks378_AA80['c-carb'] * peaks378_AA80['NAAF factor']
peaks378_AA80['moxid-NAAF80'] = peaks378_AA80['m-oxid'] * peaks378_AA80['NAAF factor']
peaks378_AA80['ndeam-NAAF80'] = peaks378_AA80['n-deam'] * peaks378_AA80['NAAF factor']
peaks378_AA80['qdeam-NAAF80'] = peaks378_AA80['q-deam'] * peaks378_AA80['NAAF factor']
peaks378_AA80['khydr-NAAF80'] = peaks378_AA80['k-hydr'] * peaks378_AA80['NAAF factor']
peaks378_AA80['rmeth-NAAF80'] = peaks378_AA80['r-meth'] * peaks378_AA80['NAAF factor']

# write the dataframe to a new csv
peaks378_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN80_NAAF.csv")

peaks378_AA80.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,T-NAAF80,V-NAAF80,W-NAAF80,Y-NAAF80,ccarb-NAAF80,moxid-NAAF80,ndeam-NAAF80,qdeam-NAAF80,khydr-NAAF80,rmeth-NAAF80
0,VVEEAPK,0.259282,1,0,0,2,0,0,0,0,...,0.000000,0.518563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SLEAESR,0.166264,1,0,0,2,0,0,0,0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TEEAVEEK,0.065793,1,0,0,4,0,0,0,0,...,0.065793,0.065793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EEDLER,0.092639,0,0,1,3,0,0,0,0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FEELVEK,0.264143,0,0,0,3,1,0,0,0,...,0.000000,0.264143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet378 PeaksDN results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaks378_AA80['NAAF factor'].sum(),
        'A-NAAF': peaks378_AA80['A-NAAF80'].sum(),
        'C-NAAF': peaks378_AA80['C-NAAF80'].sum(),
        'D-NAAF': peaks378_AA80['D-NAAF80'].sum(),
        'E-NAAF': peaks378_AA80['E-NAAF80'].sum(),
        'F-NAAF': peaks378_AA80['F-NAAF80'].sum(),
        'G-NAAF': peaks378_AA80['G-NAAF80'].sum(),
        'H-NAAF': peaks378_AA80['H-NAAF80'].sum(),
        'I-NAAF': peaks378_AA80['I-NAAF80'].sum(),
        'K-NAAF': peaks378_AA80['K-NAAF80'].sum(),
        'L-NAAF': peaks378_AA80['L-NAAF80'].sum(),
        'M-NAAF': peaks378_AA80['M-NAAF80'].sum(),
        'N-NAAF': peaks378_AA80['N-NAAF80'].sum(),
        'P-NAAF': peaks378_AA80['P-NAAF80'].sum(),
        'Q-NAAF': peaks378_AA80['Q-NAAF80'].sum(),
        'R-NAAF': peaks378_AA80['R-NAAF80'].sum(),
        'S-NAAF': peaks378_AA80['S-NAAF80'].sum(),
        'T-NAAF': peaks378_AA80['T-NAAF80'].sum(),
        'V-NAAF': peaks378_AA80['V-NAAF80'].sum(),
        'W-NAAF': peaks378_AA80['W-NAAF80'].sum(),
        'Y-NAAF': peaks378_AA80['Y-NAAF80'].sum(),
        'C-carb-NAAF': peaks378_AA80['ccarb-NAAF80'].sum(),
        'M-oxid-NAAF': peaks378_AA80['moxid-NAAF80'].sum(),
        'N-deam-NAAF': peaks378_AA80['ndeam-NAAF80'].sum(),
        'Q-deam-NAAF': peaks378_AA80['qdeam-NAAF80'].sum(),
        'K-hydr-NAAF': peaks378_AA80['khydr-NAAF80'].sum(),
        'R-meth-NAAF': peaks378_AA80['rmeth-NAAF80'].sum()
       }

totalpeaks378_AA80 = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF', 'K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, P, and Rs


totalpeaks378_AA80['% C w/ carb. NAAF'] = totalpeaks378_AA80['C-carb-NAAF'] / totalpeaks378_AA80['C-NAAF']
totalpeaks378_AA80['% M w/ oxid. NAAF'] = totalpeaks378_AA80['M-oxid-NAAF'] / totalpeaks378_AA80['M-NAAF']
totalpeaks378_AA80['% N w/ deam. NAAF'] = totalpeaks378_AA80['N-deam-NAAF'] / totalpeaks378_AA80['N-NAAF']
totalpeaks378_AA80['% Q w/ deam. NAAF'] = totalpeaks378_AA80['Q-deam-NAAF'] / totalpeaks378_AA80['Q-NAAF']
totalpeaks378_AA80['% K w/ hydr. NAAF'] = totalpeaks378_AA80['K-hydr-NAAF'] / totalpeaks378_AA80['K-NAAF']
totalpeaks378_AA80['% R w/ meth. NAAF'] = totalpeaks378_AA80['R-meth-NAAF'] / totalpeaks378_AA80['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks378_AA80['NAAF check'] = totalpeaks378_AA80['NAAF'] / 440778.253609

# write modified dataframe to new txt file, same name + totals
totalpeaks378_AA80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN80_NAAF_totals.csv")

totalpeaks378_AA80.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,Q-deam-NAAF,K-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,436.133155,385.415249,10.808879,48.150253,242.331706,64.274328,105.491028,2.588275,0.0,103.925942,...,5.706372,6.827074,224.348971,1.0,0.537111,0.461891,0.148267,0.065692,0.595002,0.000989


### Visualizing the results

In [15]:
print("ALC max: ", peaks378_80['ALC (%)'].max())
print("ALC min: ", peaks378_80['ALC (%)'].min())

ALC max:  99
ALC min:  80


### Exporting txt files of stripped peptides at confidence cutoffs:

In [16]:
# keep only peptide column >50% ALC
pep378_50_moddup = peaks378_50[["Peptide"]]

# keep only the stripped peptide column for the >50% ALC
# this is what we'll use for UniPept input, etc
pep378_50_dup = peaks378_50[["stripped peptide"]]

# deduplicate both of these lists
pep378_50_mod = pep378_50_moddup.drop_duplicates()
pep378_50 = pep378_50_dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified peptides in 378:", len(pep378_50_moddup))
print("Deduplicated modified peptides in 378:", len(pep378_50_mod))
print("Total strippled peptides in 378:", len(pep378_50_dup))
print("Deduplicated stripped peptides in 378:", len(pep378_50))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep378_50.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN50_stripped_peptides.fas


Total modified peptides in 378: 1575
Deduplicated modified peptides in 378: 1564
Total strippled peptides in 378: 1575
Deduplicated stripped peptides in 378: 1528


In [17]:
# keep only peptide column >80% ALC
pep378_80_moddup = peaks378_80[["Peptide"]]

# keep only the stripped peptide column for the >80% ALC
# this is what we'll use for UniPept input, etc
pep378_80_dup = peaks378_80[["stripped peptide"]]

# deduplicate both of these lists
pep378_80_mod = pep378_80_moddup.drop_duplicates()
pep378_80 = pep378_80_dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified peptides in 378:", len(pep378_80_moddup))
print("Deduplicated modified peptides in 378:", len(pep378_80_mod))
print("Total strippled peptides in 378:", len(pep378_80_dup))
print("Deduplicated stripped peptides in 378:", len(pep378_80))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep378_80.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN80_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 

!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN80_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PeaksDN-PTMopt/378/ETNP-SKQ17-378-100m-trap_PTMopt_15ppm_DN80_stripped_peptides.fas


Total modified peptides in 378: 459
Deduplicated modified peptides in 378: 450
Total strippled peptides in 378: 459
Deduplicated stripped peptides in 378: 448
